In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
import cPickle as pickle
import html5lib
import os
import ipyparallel as ipp

from bs4 import BeautifulSoup as bs
from urllib2 import urlopen
from urllib2 import Request
from urllib2 import URLError

reload(sys)
sys.setdefaultencoding('utf-8')
sys.setrecursionlimit(50000)

%matplotlib inline

In [2]:
#Corremos el cluster
rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

# Vamos a cargar los datos que realmente vamos a usar, y mergeo los que yo tengo para ver que cantidad real de datos tenemos

In [4]:
realData = pickle.load(open("../../../datos/fullPriceM2.p","rb"))

In [5]:
data = pickle.load(open("../../../datos/dataframe3.p","rb"))

In [10]:
realData.head(5)

id  created_on operation  \
0  c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2  2012-09-19      sell   
1  a205c53378e7879cf7519659317fb410c060d20e  2012-10-22      sell   
2  9446052c949050359fb08b9f1ba83ba2eaa20c99  2012-12-08      sell   
3  4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c  2013-01-04      sell   
5  61f3789618edabeaa7b5fc06ff36112f8a06557f  2013-01-26      sell   

  property_type            place_name  \
0         house  San Antonio De Padua   
1         house  San Antonio De Padua   
2         house             Mataderos   
3     apartment                 Pilar   
5     apartment                 Pilar   

                             place_with_parent_names country_name  \
0  |Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...    Argentina   
1  |Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...    Argentina   
2              |Argentina|Capital Federal|Mataderos|    Argentina   
3         |Argentina|Bs.As. G.B.A. Zona Norte|Pilar|    Argentina   
5         |Argentina|Bs.As. G.B.A. Zona Norte|Pilar|    Argentina   

                 state_name  geonames_id                        lat-lon  \
0  Bs.As. G.B.A. Zona Oeste    3429244.0                            NaN   
1  Bs.As. G.B.A. Zona Oeste    3429244.0                            NaN   
2           Capital Federal    3430787.0  -34.6544492346,-58.4895603251   
3  Bs.As. G.B.A. Zona Norte    3429979.0          -34.429071,-58.795418   
5  Bs.As. G.B.A. Zona Norte    3429979.0          -34.439241,-58.798589   

                 ...                 price_usd_per_m2  price_per_m2  floor  \
0                ...                       360.360360   1290.322581    NaN   
1                ...                      1000.000000   1000.000000    NaN   
2                ...                       616.666667   1796.116505    NaN   
3                ...                      1796.296296   1796.296296    NaN   
5                ...                      2222.222222   2222.222222    NaN   

  rooms  expenses                                      properati_url  \
0   NaN       NaN  http://www.properati.com.ar/udh_venta_casa_mer...   
1   NaN       NaN  http://www.properati.com.ar/20hi_venta_casa_me...   
2   5.0       NaN  http://www.properati.com.ar/4m4m_venta_casa_ma...   
3   2.0       NaN  http://www.properati.com.ar/518a_venta_departa...   
5   2.0       NaN  http://www.properati.com.ar/59ec_venta_departa...   

                                         description  \
0  CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...   
1  CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...   
2  Casa Multifamiliar en Mataderos!!! Sobre lote ...   
3  Venta - Departamento - Argentina, Buenos Aires...   
5  Venta - Departamento - Argentina, Buenos Aires...   

                                             title  \
0                                    CASA EN VENTA   
1                                    CASA EN VENTA   
2                                    CASA EN VENTA   
3  Departamento en Venta ubicado en Arcos de Pilar   
5           Departamento en Venta ubicado en Pilar   

                                     image_thumbnail  \
0  https://thumbs4.properati.com/1/ufoyqwscVk8n3e...   
1  https://thumbs4.properati.com/5/I5RS_jAIUseo9H...   
2  https://thumbs4.properati.com/6/DsNJjCb5_zIPV0...   
3  https://thumbs4.properati.com/8/bJn9FqcBRGi_8H...   
5  https://thumbs4.properati.com/5/hiXoFGFHBiIhV2...   

                             latlon  
0            (-34.66692, -58.70097)  
1            (-34.66692, -58.70097)  
2  (-34.6544492346, -58.4895603251)  
3          (-34.429071, -58.795418)  
5          (-34.439241, -58.798589)  

[5 rows x 28 columns]

In [11]:
realData.info()

In [14]:
realData = realData[['id','created_on','operation','property_type','place_name','place_with_parent_names',
                     'country_name','state_name','geonames_id','lat-lon','lat','lon','price','currency',
                     'price_aprox_local_currency','price_aprox_usd','surface_total_in_m2','surface_covered_in_m2',
                     'price_usd_per_m2','price_per_m2','floor','expenses','properati_url','description',
                     'title','image_thumbnail','latlon']]


In [15]:
data.head(5)

id  \
0  c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2   
1  a205c53378e7879cf7519659317fb410c060d20e   
4  9446052c949050359fb08b9f1ba83ba2eaa20c99   
5  4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c   
6  535441f569473f6e5d687c38b27d9b9c6e69e284   

                                         description  \
0  CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...   
1  CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...   
4  Casa Multifamiliar en Mataderos!!! Sobre lote ...   
5  Venta - Departamento - Argentina, Buenos Aires...   
6  Venta - Local - Argentina, Buenos Aires, Zona ...   

                                             title  \
0                                    CASA EN VENTA   
1                                    CASA EN VENTA   
4                                    CASA EN VENTA   
5  Departamento en Venta ubicado en Arcos de Pilar   
6             Local en Venta ubicado en San Isidro   

                                       properati_url  rooms  ambientes  \
0  http://www.properati.com.ar/udh_venta_casa_mer...    NaN        NaN   
1  http://www.properati.com.ar/20hi_venta_casa_me...    3.0        4.0   
4  http://www.properati.com.ar/4m4m_venta_casa_ma...    5.0        3.0   
5  http://www.properati.com.ar/518a_venta_departa...    2.0        2.0   
6  http://www.properati.com.ar/52p4_venta_local_s...    1.0        NaN   

                                                webp  banios  
0  <!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...     2.0  
1                                               None     NaN  
4                                               None     NaN  
5                                               None     NaN  
6                                               None     NaN

In [16]:
data = data[['id','rooms','ambientes','banios','webp']]

# Hacemos un merge, dejando todas las claves que esten en real

In [17]:
total = pd.merge(realData, data, on='id', how='left')

In [19]:
total.head(5)

id  created_on operation  \
0  c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2  2012-09-19      sell   
1  a205c53378e7879cf7519659317fb410c060d20e  2012-10-22      sell   
2  9446052c949050359fb08b9f1ba83ba2eaa20c99  2012-12-08      sell   
3  4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c  2013-01-04      sell   
4  61f3789618edabeaa7b5fc06ff36112f8a06557f  2013-01-26      sell   

  property_type            place_name  \
0         house  San Antonio De Padua   
1         house  San Antonio De Padua   
2         house             Mataderos   
3     apartment                 Pilar   
4     apartment                 Pilar   

                             place_with_parent_names country_name  \
0  |Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...    Argentina   
1  |Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...    Argentina   
2              |Argentina|Capital Federal|Mataderos|    Argentina   
3         |Argentina|Bs.As. G.B.A. Zona Norte|Pilar|    Argentina   
4         |Argentina|Bs.As. G.B.A. Zona Norte|Pilar|    Argentina   

                 state_name  geonames_id                        lat-lon  \
0  Bs.As. G.B.A. Zona Oeste    3429244.0                            NaN   
1  Bs.As. G.B.A. Zona Oeste    3429244.0                            NaN   
2           Capital Federal    3430787.0  -34.6544492346,-58.4895603251   
3  Bs.As. G.B.A. Zona Norte    3429979.0          -34.429071,-58.795418   
4  Bs.As. G.B.A. Zona Norte    3429979.0          -34.439241,-58.798589   

                         ...                          expenses  \
0                        ...                               NaN   
1                        ...                               NaN   
2                        ...                               NaN   
3                        ...                               NaN   
4                        ...                               NaN   

                                       properati_url  \
0  http://www.properati.com.ar/udh_venta_casa_mer...   
1  http://www.properati.com.ar/20hi_venta_casa_me...   
2  http://www.properati.com.ar/4m4m_venta_casa_ma...   
3  http://www.properati.com.ar/518a_venta_departa...   
4  http://www.properati.com.ar/59ec_venta_departa...   

                                         description  \
0  CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...   
1  CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...   
2  Casa Multifamiliar en Mataderos!!! Sobre lote ...   
3  Venta - Departamento - Argentina, Buenos Aires...   
4  Venta - Departamento - Argentina, Buenos Aires...   

                                             title  \
0                                    CASA EN VENTA   
1                                    CASA EN VENTA   
2                                    CASA EN VENTA   
3  Departamento en Venta ubicado en Arcos de Pilar   
4           Departamento en Venta ubicado en Pilar   

                                     image_thumbnail  \
0  https://thumbs4.properati.com/1/ufoyqwscVk8n3e...   
1  https://thumbs4.properati.com/5/I5RS_jAIUseo9H...   
2  https://thumbs4.properati.com/6/DsNJjCb5_zIPV0...   
3  https://thumbs4.properati.com/8/bJn9FqcBRGi_8H...   
4  https://thumbs4.properati.com/5/hiXoFGFHBiIhV2...   

                             latlon  rooms  ambientes  banios  \
0            (-34.66692, -58.70097)    NaN        NaN     2.0   
1            (-34.66692, -58.70097)    3.0        4.0     NaN   
2  (-34.6544492346, -58.4895603251)    5.0        3.0     NaN   
3          (-34.429071, -58.795418)    2.0        2.0     NaN   
4          (-34.439241, -58.798589)    2.0        NaN     NaN   

                                                webp  
0  <!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...  
1                                               None  
2                                               None  
3                                               None  
4                                               None  

[5 rows x 31 columns]

In [20]:
total.info()

In [21]:
pickle.dump(total,open("DataFrame.p","wb"))